In [1]:
%use dataframe
%use multik

In [2]:
import kotlin.math.ln
import kotlin.math.min
import kotlin.math.exp
import kotlin.random.Random

In [3]:
val df = dataFrameOf(
    "X1" to List(100) { 1 },
    "X2" to List(100) { if (it<50) 1 else 0 }
).add("y") {
    val p= 1/(1+exp(0.5*"X1"<Int>().toFloat() - 0.5*"X2"<Int>().toFloat()))
    if (Random.nextFloat()<p) 1 else 0
}
df.head(10)

X1,X2,y
1,1,1
1,1,1
1,1,1
1,1,1
1,1,1
1,1,0
1,1,0
1,1,1
1,1,1
1,1,1


In [116]:
df.writeCSV("testdata.csv")

In [6]:
val totals = df2.groupBy { group }.aggregate { count() into "total" }

In [7]:
totals

group,total
0,50
1,50


In [19]:
totals.filter { group == 1 }.total.toList()

[50]

In [4]:
val theta: (beta: NDArray<Double, D2>, X: NDArray<Double, D2>) -> Double =  { beta,X ->  X.dot(beta)[0,0] }
val dtheta: (beta: NDArray<Double, D2>, X: NDArray<Double, D2>) -> NDArray<Double, D2> =  { beta,X ->  X }
val ddtheta: (beta: NDArray<Double, D2>, X: NDArray<Double, D2>) -> NDArray<Double, D2> =  { beta,X ->  mk.zeros<Double>(beta.shape[0], beta.shape[0]) }

val b: (theta: Double) -> Double =  { theta ->  ln(1+exp(theta)) }
val db: (theta: Double) -> Double =  { theta ->  exp(theta)/(1+exp(theta)) }
val ddb: (theta: Double) -> Double =  { theta ->  exp(theta)/((1+exp(theta)).pow(2.0)) }

In [62]:
val theta: (beta: DoubleArray, X: DoubleArray) -> Double =  { beta,X ->  beta.foldIndexed(initial=0.0){i,acc,el -> acc + el*X[i] } }
val dtheta: (beta: DoubleArray, X: DoubleArray) -> DoubleArray =  { beta,X ->  X }
val ddtheta: (beta: DoubleArray, X: DoubleArray) -> DoubleArray =  { beta,X ->  DoubleArray(beta.size * beta.size) {0.0} }

val b: (theta: Double) -> Double =  { theta ->  ln(1+exp(theta)) }
val db: (theta: Double) -> Double =  { theta ->  exp(theta)/(1+exp(theta)) }
val ddb: (theta: Double) -> Double =  { theta ->  exp(theta)/((1+exp(theta)).pow(2.0)) }

In [59]:
val theta: (beta: NDArray<Double, D2>, X: NDArray<Double, D2>) -> Double =  { beta,X ->  X.dot(beta)[0,0] }

In [61]:
mk.ones<Double>(3, 4)

[[1.0, 1.0, 1.0, 1.0],
[1.0, 1.0, 1.0, 1.0],
[1.0, 1.0, 1.0, 1.0]]

In [46]:
df.sumOf { (y.toDouble() -db(theta(0.0))) *dtheta(0.0) }

23.0

In [14]:
val zeros = mk.zeros<Double>(3, 4)

In [129]:
val d = mk.ndarray(doubleArrayOf(1.0, 1.0, 3.0, 4.0), 2, 2)

In [ ]:
NDArray<Double, D2>

In [33]:
mk.linalg.inv(d)

[[3.9999998, -0.9999999],
[-2.9999998, 0.99999994]]

In [5]:
df

X1,X2,y
1,1,1
1,1,1
1,1,1
1,1,1
1,1,1
1,1,0
1,1,0
1,1,1
1,1,1
1,1,1


In [126]:
mk.ndarray(dtheta(doubleArrayOf(1.0,1.0),doubleArrayOf(1.0,1.0)),2,1)

[[1.0],
[1.0]]

In [130]:
dtheta(mk.zeros<Double>(2,1),mk.ndarray(doubleArrayOf(1.0,1.0),1,2))

[[1.0, 1.0]]

In [136]:
mk.ndarray(doubleArrayOf(1.0,1.0),1,2)

[[1.0, 1.0]]

In [124]:
val y=mk.ndarray(df.y.toList().map { it.toDouble()} ,df.count(),1)
val df2=df.add("X") {
  doubleArrayOf(X1.toDouble(),X2.toDouble())
}.remove {X1 and X2}

In [29]:
mk.ndarray(df2.map { db(theta(mk.ndarray(doubleArrayOf(1.0,0.0),2,1), it.X)) },df2.count(),1)

Line_38.jupyter.kts (1:31 - 69) Type mismatch: inferred type is D2Array<Double> /* = NDArray<Double, D2> */ but DoubleArray was expected

In [36]:
//mk.ndarray(df2.map{ dtheta(mk.ndarray(doubleArrayOf(1.0,0.0),2,1), it.X).toList() })
(y.elementMinus((df2.map { db(theta(mk.ndarray(doubleArrayOf(1.0,0.0),2,1), it.X)) },df2.count(),1)).transpose()
    .dot(mk.ndarray(df2.map{ dtheta(mk.ndarray(doubleArrayOf(1.0,0.0),2,1), it.X).toList() }))

Line_47.jupyter.kts (2:85 - 85) Expecting ')'

In [57]:
//array version

residual.transpose().dot(mk.ndarray(df2.map{ dtheta(doubleArrayOf(1.0,0.0), it.X) }.toTypedArray<DoubleArray>()))
    /*.transpose()
    .dot(mk.ndarray(df2.map{ dtheta(mk.ndarray(doubleArrayOf(1.0,0.0),2,1), it.X).toList() }))*/

[[-24.1058578630005, -3.5529289315002424]]

In [130]:
df2.sumOf { b(theta(beta, it.X)) + y*theta(beta, it.X) }

83.05196744571256

In [98]:
residual.transpose().dot(mk.ndarray(df2.map{ dtheta(beta, it.X) }.toTypedArray<DoubleArray>()))

[[-24.1058578630005, -3.5529289315002424]]

In [92]:
mk.linalg.inv(residual.transpose().dot(mk.ndarray(df2.map{ ddtheta(beta, it.X) }.toTypedArray<DoubleArray>())).reshape(beta.size,beta.size)
-
mk.ndarray(df2.map { ddb(theta(beta, it.X)) },df2.count(), 1).transpose().dot(mk.ndarray(df2.map { 
    val ddb_id = dtheta(beta, it.X)
    DoubleArray(beta.size*beta.size) {
        ddb_id[it % beta.size] * ddb_id[it / beta.size]
    }
}.toTypedArray<DoubleArray>())).reshape(beta.size,beta.size))

[[-0.10172322539260972, 0.10172322539260972],
[0.10172322539260972, -0.2034464507852195]]

In [134]:
var beta=doubleArrayOf(1.0,0.0)
repeat(10) {
    val l = df2.sumOf { ln(b(theta(beta, it.X))) + y*theta(beta, it.X) }
    println("likelihood: $l")
    val residual = (y - mk.ndarray(df2.map { db(theta(beta, it.X)) },df2.count(),1))
    val dl = residual.transpose().dot(mk.ndarray(df2.map{ dtheta(beta, it.X) }.toTypedArray<DoubleArray>()))
    val hessian = mk.linalg.inv(residual.transpose().dot(mk.ndarray(df2.map{ ddtheta(beta, it.X) }.toTypedArray<DoubleArray>())).reshape(beta.size,beta.size)
-
mk.ndarray(df2.map { ddb(theta(beta, it.X)) },df2.count(), 1).transpose().dot(mk.ndarray(df2.map { 
    val ddb_id = dtheta(beta, it.X)
    DoubleArray(beta.size*beta.size) {
        ddb_id[it % beta.size] * ddb_id[it / beta.size]
    }
}.toTypedArray<DoubleArray>())).reshape(beta.size,beta.size))
    beta = beta.elementMinus(dl.dot(hessian).toDoubleArray())
    
    println(beta.joinToString())
}

likelihood: 76.25138805025851
-1.09071022217729, 1.7292948316743668
likelihood: -55.29532333679605
-0.7267312095761589, 1.3899302462059988
likelihood: -32.47221036101277
-0.7536434749367142, 1.416937690897621
likelihood: -34.01098394852346
-0.7537717994125047, 1.4170660168227684
likelihood: -34.01836080122053
-0.7537718023763801, 1.4170660197866447
likelihood: -34.0183609716049
-0.7537718023763802, 1.4170660197866447
likelihood: -34.01836097160495
-0.7537718023763799, 1.417066019786644
likelihood: -34.01836097160495
-0.7537718023763802, 1.4170660197866447
likelihood: -34.01836097160495
-0.7537718023763799, 1.417066019786644
likelihood: -34.01836097160495
-0.7537718023763802, 1.4170660197866447


In [51]:
inline fun <reified T> toArray(list: List<*>): Array<T> {
    return (list as List<T>).toTypedArray()
}

In [ ]:
df2.map{ddtheta(beta)

In [102]:
/*mk.ndarray(df.y.toList(),100,1)*/
mk.ndarray(db(theta(mk.zeros<Double>(2,1))))

Line_243.jupyter.kts (2:4 - 11) None of the following functions can be called with the arguments supplied: 
public fun Multik.ndarray(args: Array<ByteArray>): D2Array<Byte> /* = NDArray<Byte, D2> */ defined in org.jetbrains.kotlinx.multik.api
public fun Multik.ndarray(args: Array<DoubleArray>): D2Array<Double> /* = NDArray<Double, D2> */ defined in org.jetbrains.kotlinx.multik.api
public fun Multik.ndarray(args: Array<FloatArray>): D2Array<Float> /* = NDArray<Float, D2> */ defined in org.jetbrains.kotlinx.multik.api
public fun Multik.ndarray(args: Array<IntArray>): D2Array<Int> /* = NDArray<Int, D2> */ defined in org.jetbrains.kotlinx.multik.api
public fun Multik.ndarray(args: Array<LongArray>): D2Array<Long> /* = NDArray<Long, D2> */ defined in org.jetbrains.kotlinx.multik.api
public fun Multik.ndarray(args: Array<ShortArray>): D2Array<Short> /* = NDArray<Short, D2> */ defined in org.jetbrains.kotlinx.multik.api
public fun Multik.ndarray(args: ByteArray): D1Array<Byte> /* = NDArray<By

In [45]:
fun DataFrame.sumOf<DoubleArray>(a: {()->DoubleArray}) : DoubleArray {
    var sum = DoubleArray(a.size) {0.0}
    return sum
}

Line_186.jupyter.kts (1:37 - 37) Type expected
Line_186.jupyter.kts (1:39 - 39) Expecting an expression
Line_186.jupyter.kts (1:40 - 53) Unexpected tokens (use ';' to separate expressions on the same line)
Line_186.jupyter.kts (1:54 - 55) Expecting an element
Line_186.jupyter.kts (1:56 - 57) Expecting an element

In [39]:
df.sumOf { mk.ndarray(dtheta(doubleArrayOf(1.0,1.0),doubleArrayOf(1.0,1.0)),2,1) }

Line_180.jupyter.kts (1:12 - 81) Type mismatch: inferred type is D2Array<Double> /* = NDArray<Double, D2> */ but Number? was expected

In [48]:
df.sumOf { ( -ddb(theta(0.0))) *(dtheta(0.0).pow(2.0)) }

-25.0

In [12]:
var beta=0.0
repeat(15) {
    beta = beta - df.sumOf { (y.toDouble() - db(theta(beta))) *dtheta(beta) }/
    (df.sumOf { (y.toDouble() -db(theta(beta))) *ddtheta(beta) } + df.sumOf { ( -ddb(theta(beta))) *(dtheta(beta).pow(2.0)) })
    println(beta)
}

Line_62.jupyter.kts (3:30 - 75) Cannot access 'times': it is private in file
Line_62.jupyter.kts (3:30 - 75) Type mismatch: inferred type is DoubleArray but Number? was expected
Line_62.jupyter.kts (3:54 - 58) Type mismatch: inferred type is Double but DoubleArray was expected
Line_62.jupyter.kts (3:54 - 59) No value passed for parameter 'X'
Line_62.jupyter.kts (3:70 - 74) Type mismatch: inferred type is Double but DoubleArray was expected
Line_62.jupyter.kts (3:70 - 75) No value passed for parameter 'X'
Line_62.jupyter.kts (3:77 - 78) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public operator fun Byte.div(other: ComplexDouble): ComplexDouble defined in org.jetbrains.kotlinx.multik.ndarray.complex
public operator fun Byte.div(other: ComplexFloat): ComplexFloat defined in org.jetbrains.kotlinx.multik.ndarray.complex
public operator fun <D : Dimension> Byte.div(other: MultiArray<Byte, TypeVariable(D)>): NDArray<Byte, TypeVaria

In [30]:
fun DoubleArray.elementPlus(other: DoubleArray): DoubleArray {
    return ( DoubleArray(this.size) { it -> this[it] + other [it]})
}
fun DoubleArray.elementMinus(other: DoubleArray): DoubleArray {
    return ( DoubleArray(this.size) { it -> this[it] - other [it]})
}

In [26]:
inline fun <T> Iterable<T>.sumOfArray(selector: (T) -> DoubleArray): DoubleArray {
   
    if (this.none()) return DoubleArray(1) { 0.0 }
    //return DoubleArray(1) {0.0}
    var sum = DoubleArray(selector(this.first()).size) { 0.0 }
    for (element in this) {
        //sum = sum.elementPlus(selector(element))    
        for (i in sum.indices) {
            sum[i] += selector(element)[i]
        }
    }
    return sum
}

In [27]:
List(4) { it1 ->
 DoubleArray(3) { it2 -> it2.toDouble() } 
}.sumOfArray { it }


[0.0, 4.0, 8.0]

In [15]:
DoubleArray(3) { it2 -> it2.toDouble() } + DoubleArray(3) { it2 -> it2.toDouble() } 

[0.0, 2.0, 4.0]